In [2]:
import numpy as np
from tqdm import tqdm
import wandb as wb
import imageio
from io import BytesIO
import pickle
import zarr
from pathlib import Path
from src.visualization.render_mp4 import create_mp4

In [3]:
np_images = np.linspace(0, 256, 750, dtype=np.uint8).reshape(750, 1, 1, 1)

# Repeat this (750,) to (750, 720, 1280, 3)
np_images = np.repeat(np_images, 720, axis=1)
np_images = np.repeat(np_images, 1280, axis=2)
np_images = np.repeat(np_images, 3, axis=3)

np_images.shape

(750, 720, 1280, 3)

In [5]:
# For MP4
create_mp4(np_images, "output.mp4")

  0%|          | 2/750 [00:00<00:42, 17.75it/s]

100%|██████████| 750/750 [00:03<00:00, 211.11it/s]

File saved as output.mp4


In [15]:
np_images = np.linspace(0, 256, 750, dtype=np.uint8).reshape(750, 1, 1, 1)

# Repeat this (750,) to (750, 720, 1280, 3)
np_images = np.repeat(np_images, 224, axis=1)
np_images = np.repeat(np_images, 224, axis=2)
np_images = np.repeat(np_images, 3, axis=3)

np_images.shape

(750, 224, 224, 3)

In [16]:
create_gif_or_mp4(np_images, "output_small.mp4", is_mp4=True)

100%|██████████| 750/750 [00:00<00:00, 3374.72it/s]

File saved as output_small.mp4


In [3]:
def create_in_memory_mp4(np_images, fps=10):
    output = BytesIO()

    writer_options = {"fps": fps}
    writer_options["format"] = "mp4"
    writer_options["codec"] = "libx264"
    writer_options["pixelformat"] = "yuv420p"

    with imageio.get_writer(output, **writer_options) as writer:
        for img in tqdm(np_images):
            writer.append_data(img)
    output.seek(0)
    return output

In [26]:
# Keep black
vid1 = np_images.copy()

# Make Red
vid2 = np_images.copy()
vid2[:, :, :, 1] = 0
vid2[:, :, :, 2] = 0

# Make Green
vid3 = np_images.copy()
vid3[:, :, :, 0] = 0
vid3[:, :, :, 2] = 0

# Make Blue
vid4 = np_images.copy()
vid4[:, :, :, 0] = 0
vid4[:, :, :, 1] = 0

vids = [vid1, vid2, vid3, vid4]

In [33]:
import zarr

data = zarr.open(
    "/data/scratch/ankile/furniture-data-old/data/processed/sim/image_small/one_leg/data.zarr"
)

vid5 = data["color_image2"][:750]

In [34]:
vids.append(vid5)

In [35]:
wb.init(project="test-video")

# Make a table object
table = wb.Table(columns=["video"])

# Add the videos to the table
for vid in vids:
    video = create_in_memory_mp4(vid, fps=10)
    table.add_data(wb.Video(video, fps=10, format="mp4"))

# Log the table
wb.log({"videos": table})

# end the run
wb.finish()

100%|██████████| 750/750 [00:00<00:00, 1734.42it/s]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


## Check the rollouts produced and saved

In [19]:
rollout_path = "/data/pulkitag/data/ankile/furniture-data/data/raw/sim_rollouts/2024-01-03_08-23-57/rollout_15.pkl"

with open(rollout_path, "rb") as f:
    rollout = pickle.load(f)

rollout.keys()

dict_keys(['observations', 'actions', 'rewards', 'success', 'furniture'])

In [26]:
rollout["success"], sum(rollout["rewards"])

(False, 0.0)

In [27]:
len(rollout["observations"]), len(rollout["actions"]), len(rollout["rewards"])

(600, 600, 600)

In [21]:
# Produce a video of the rollout
vid1, vid2 = zip(
    *[(o["color_image1"], o["color_image2"]) for o in rollout["observations"]]
)

vid = np.concatenate([vid1, vid2], axis=2)

In [6]:
# make video
create_gif_or_mp4(vid, "output_rollout.mp4")

100%|██████████| 600/600 [00:00<00:00, 758.79it/s]


File saved as output_rollout.mp4


## Some unrelated testing

In [4]:
data = zarr.open(
    "/data/scratch/ankile/furniture-diffusion/replay_buffer.zarr", mode="r"
)

In [5]:
len(data["data"]["stage"][:])

27672

In [6]:
len(data["meta"]["episode_ends"])

136

In [11]:
data["data"]["timestamp"][3]

1672955667.7093015

## More Unrelated Testing: Inspect demo collected from teleop

In [7]:
pkl_file = "/data/pulkitag/data/anthony/repos/research2/furniture-bench/test/drawer/2023-12-18-17:42:56/2023-12-18-17:42:56.pkl"

In [8]:
with open(pkl_file, "rb") as f:
    rollout = pickle.load(f)

In [21]:
# Produce a video of the rollout
vid1, vid2 = zip(
    *[(o["color_image1"], o["color_image2"]) for o in rollout["observations"]]
)

vid = np.concatenate([vid1, vid2], axis=2)

# make video
create_gif_or_mp4(vid, "output_teleop.mp4")

100%|██████████| 1297/1297 [00:01<00:00, 1113.30it/s]

File saved as output_teleop.mp4
